# Réaliser un traitement dans un environnement Big Data sur le Cloud
## Notebook 2 - Traitement des données sur Amazon Web Services - Elastic MapReduce  
### Version simplifiée

## Sommaire  
__1. Préambule__  
&emsp;1.1 Contexte  
&emsp;1.2 Étapes du projet   
&emsp;1.3 Remarques   
__2. Exécution du code__  
&emsp;2.1 Démarrage de la session Spark  
&emsp;2.2 Installation des packages  
&emsp;2.3 Définition des chemins d'accès aux fichiers  
&emsp;2.4 Traitement des données  
&emsp;&emsp;2.4.1 Chargement des données  
&emsp;&emsp;2.4.2 Préparation du modèle  
&emsp;&emsp;2.4.3 Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF  
&emsp;&emsp;2.4.4 Exécution des actions d'extraction de features  
&emsp;2.5 Réduction de dimension  
&emsp;&emsp;2.5.1 Mise à l'échelle : StandardScaler  
&emsp;&emsp;2.5.2 Réduction de dimension : PCA  
&emsp;2.6 Chargement des données enregistrées et validation du résultat  

# 1. Préambule  
## 1.1 Contexte  
__"Fruits!"__, très jeune start-up de l'AgriTech, cherche à proposer des solutions innovantes pour la récolte des fruits et a pour projet de développer des robots cueilleurs intelligents.  
Pour se faire connaître, elle souhaite mettre à disposition du grand public une application mobile qui permettrait aux utilisateurs de prendre en photo un fruit et d'obtenir des informations sur ce fruit.  
Une première approche d'extraction des features d'images de fruits via le modèle ModeleNetV2 a été testée en environnement local puis AWS EMR.  
## 1.2 Étapes du projet  
- Choix techniques  
- Traitement des données en environnement local (notebook 1)  
- Mise en place de l'environnement AWS EMR  
- __Traitement des données sur AWS (ce notebook)__
- Résiliation du cluster EMR
- Conclusion  
## 1.3 Remarques  
- Rendons à César ce qui appartient à César : la grande majorité du code qui suit a été repris de la première approche fournie
- Quels sont les apports par rapport à la version fournie ?  
    - Modification de la définition du décorateur `@pandas_udf` et ajout des types de données à la fonction `featurize_udf` pour s'adapter au document [SPARK-28264](https://issues.apache.org/jira/browse/SPARK-28264) et supprimer le warning associé  
    - Ajout en pyspark d'une mise à l'échelle (StandardScaler) et d'une réduction de dimension (PCA) des features après extraction
    - ⚠️ Simplification du process de préparation du modèle :
       - Chargement direct du modèle préentraîné imagenet sans la dernière couche
       - Suppression de l'étape de gel des poids préentraînés
       - Suppression du broadcasting des poids

# 2. Exécution du code  

Ce notebook est conçu pour être exécuté sur __JupyterHub__ avec un kernel __pyspark__.  
La version JupyterHub fournie par Amazon EMR est configurée avec [__Sparkmagic__](https://github.com/jupyter-incubator/sparkmagic/blob/master/README.md) qui implémente de nombreuses fonctionnalités (liste et exemples sur le [GitHub SparkMagic](https://github.com/jupyter-incubator/sparkmagic/blob/master/examples/Pyspark%20Kernel.ipynb))  
Ce notebook utilise les suivantes :  
- A l'exécution de la première cellule de code :
    - Création d'un SparkContext accessible via la variable `sc`
    - Démarrage d'une SparkSession accessible via la variable `spark`
- PySpark magics :
   - `%%info` : informations sur la session  
   - `%%pretty` : formatage HTML des dataframes Spark

## 2.1 Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725966056338_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des information sur la session en cours :

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725966056338_0002,pyspark,idle,Link,Link,None,✔


## 2.2 Import des librairies  
Les packages ont été installés à l'instanciation du cluster, via le fichier __boostrap-emr.sh__

In [3]:
import pandas as pd
import numpy as np
import sys
import io
import os
from PIL import Image
from typing import Iterator

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

import pyspark
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split, udf #, expr, slice
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler, PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des versions utilisées :

In [4]:
# Versions
print('Version python :')
print('Python : ' + sys.version)
print('\nVersions librairies :')
print('tensorflow : ' + tf.__version__)
print('pyspark : ' + pyspark.__version__)
print('PIL : ' + Image.__version__)
print('Numpy : ' + np.__version__)
print('Pandas : ' + pd.__version__)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Version python :
Python : 3.9.16 (main, Jul  5 2024, 00:00:00) 
[GCC 11.4.1 20230605 (Red Hat 11.4.1-2)]

Versions librairies :
tensorflow : 2.17.0
pyspark : 3.5.1+amzn.0.dev0
PIL : 10.4.0
Numpy : 1.26.4
Pandas : 1.3.5

## 2.3 Définition des chemins d'accès aux fichiers  
Les __22 688__ images du jeu de données Test seront utilisées, elles sont stockées dans le dossier __img__ du bucket S3 __cap9-fruits__  
Une fois extraites, les caractéristiques vectorisées des images seront stockées dans le dossier __Results__ du même bucket __cap9-fruits__  

In [5]:
PATH = 's3://cap9-fruits'
PATH_Data = PATH+'/img'
PATH_Result = PATH+'/Results_1'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://cap9-fruits
PATH_Data:   s3://cap9-fruits/img
PATH_Result: s3://cap9-fruits/Results_1

## 2.4 Traitement des données  
### 2.4.1 Chargement des données

Structure du dossier __img__ :  
img  
&emsp;└ Apple Braeburn  
&emsp;&emsp;&emsp;&emsp;└ 3_100.jpg  
&emsp;&emsp;&emsp;&emsp;└ 4_100.jpg  
&emsp;&emsp;&emsp;&emsp;└ 5_100.jpg  
&emsp;&emsp;&emsp;&emsp;└ ...  
&emsp;└ Apple Crimson Snow  
&emsp;&emsp;&emsp;&emsp;└ 80_100.jpg  
&emsp;&emsp;&emsp;&emsp;└ 81_100.jpg  
&emsp;&emsp;&emsp;&emsp;└ ...  
&emsp;└ ...  
<br>
<u>Chargement dans un dataframe Spark des fichiers __*.jpg__ au format __binaire__, avec l'option `recursiveFileLookup` pour prendre en compte tous les sous-dossiers :

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Ajout des labels au dataframe, par récupération de l'avant-dernier élément de la chaîne du path splittée par '/' :

In [7]:
%%pretty
images = images.withColumn('label', element_at(split(images['path'], '/'), -2))
images.printSchema()
images.select('path','label').show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path,label
s3://cap9-fruits/img/Watermelon/r_106_100.jpg,Watermelon
s3://cap9-fruits/img/Watermelon/r_109_100.jpg,Watermelon
s3://cap9-fruits/img/Watermelon/r_108_100.jpg,Watermelon
s3://cap9-fruits/img/Watermelon/r_107_100.jpg,Watermelon
s3://cap9-fruits/img/Watermelon/r_95_100.jpg,Watermelon


### 2.4.2 Préparation du modèle

Le modèle __MobileNetV2__ a été sélectionné pour extraire les features des images, pour sa rapidité d'exécution en comparaison avec d'autres modèles comme VGG16.  
Ce modèle est utilisé ici après un pré-entraînement sur les images issue de la base __Imagenet__ (transfer learning), et attend en entrée des images au format 224 x 224, alors que les images du jeu de données Test sont de dimension 100 x 100.  

Schéma de l'architecture globale de __MobileNetV2__ :  
![Architecture modèle MobileNetV2](https://miro.medium.com/v2/resize:fit:640/format:webp/1*5iA55983nBMlQn9f6ICxKg.png)  
<br>
La dernière couche sert à classer les images selon 1 000 catégories, elle ne sera pas utilisée ici. La couche précédente a en sortie un vecteur de dimension 1280 qui sera utilisé plus tard pour reconnaître les différents fruits via un modèle de classification.  

Les paramètres du modèle (voir [documentation MobileNet Tensorflow Keras](https://keras.io/api/applications/mobilenet/)) permettent de manière très simple :  
- de charger le modèle avec les poids pré-entraînés 'imagenet'  
- de spécifier l'exclusion de la dernière couche  
- de préciser une sortie avec un "pooling average" (plutôt qu'un tensor 4D de la dernière couche convolutionnelle)
- de spécifier le format des images en entrée

<u>Code de la création du modèle :</u>
```python
model = MobileNetV2(
    weights='imagenet',         # optionnel (valeur par défaut)
    include_top=False,          # la dernière couche n'est pas incluse
    pooling="avg",              # couche de sortie
    input_shape=(224, 224, 3)   # optionnel (valeur par défaut)
)
```

### 2.4.3 Définition du processus de chargement des images et application de leur featurisation à travers l'utilisation de pandas UDF  

![Encapsulation des appels de fonctions](encapsulation_simplifiee.png)  

<u>Encapsulation des appels de fonctions dans le processus d'extraction des caractéristiques d'images</u><br><br>
La fonction <code>featurize_udf</code> est une Pandas UDF (User Defined Function) qui encapsule tout le processus, depuis le redimensionnement des images via la fonction <code>preprocess</code> jusqu'à l'extraction des vecteurs de caractéristiques via <code>featurize_series</code>.<br>
L'intérêt de cette encapsulation dans une Pandas UDF est que Spark exécutera cette fonction sur les données distribuées, en traitant chaque partition de données dans le cluster de manière parallèle.


<u>Fonction de redimensionnent des images au format 224 x 224 :</u>  

In [8]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    # Chargement et redimensionnement 224 x 224
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    # Utilisation de la fonction 'img_to_array' de Tensorflow Keras
    # pour coller au format d'input attendu par le modèle
    arr = img_to_array(img)
    # Normalisation
    return preprocess_input(arr)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Fonction d'extraction des vecteurs des caractéristiques des images (feature extraction) :

In [9]:
def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    # Preprocess de chaque image puis stockage dans
    # un tableau numpy 4D (nombre d'images, hauteur, largeur, canal RVB)
    input = np.stack(content_series.map(preprocess))
    # Récupération des vecteurs des caractéristiques des images
    preds = model.predict(input)
    # Aplatissement des vecteurs
    output = [p.flatten() for p in preds]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Une __UDF (User Defined Function)__ est une 'fonction définie par l'utilisateur' qui permet d'exécuter des opérations personnalisées sur les données dans un DataFrame Spark. On utilise une UDF pour appliquer des transformations ou des calculs qui ne sont pas disponibles nativement dans Spark SQL.  
Les __Pandas UDFs__ sont des UDFs permettant d'utiliser des fonctions pandas sur des dataframes Spark, en tirant parti des optimisations vectorisées de pandas pour améliorer les performances par rapport aux UDFs de base. Il y a 3 types de pandas UDFs :  
- __Scalar UDF__ : fonctionne sur une série pandas et renvoie une autre série pandas
- __Grouped Mad UDF__ : appliquée à un groupe de données, renvoie un DataFrame pandas
- __Iterator UDF__ : prend un itérateur de séries pandas en entrée et renvoie un itérateur de séries pandas en sortie, ce qui permet un traitement des données par lots.  

<u>Iterator UDF pandas permettant l'extraction des caractéristiques des images par lots :

In [10]:
# Décorateur indiquant que la fonction 'featurize_udf' est une pandas UDF renvoyant un tableau de floats
@pandas_udf(ArrayType(FloatType()))
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    '''
    Cette fonction est une UDF pandas Iterator qui encapsule la fonction de featurization.
    '''
    # Chargement et initialisation du modèle
    model = MobileNetV2(
        weights='imagenet',         # optionnel (valeur par défaut)
        include_top=False,          # la dernière couche n'est pas incluse
        pooling="avg",              # couche de sortie
        input_shape=(224, 224, 3)   # optionnel (valeur par défaut)
    )
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2.4.4 Exécution des actions d'extraction de features

L'intérêt d'utiliser Spark et les pandas UDFs est de pouvoir paralléliser les taches.  
En partionnant le dataframe Spark contenant les images à featuriser, Spark exécutera la fonction `featurize_udf` __en parallèle sur chaque partition__.  

Les bonnes pratiques se basent sur le __nombre de coeurs__ dans le cluster et la __taille des données__. Il est préconisé :  
- de définir le nombre de partitions à 2 à 4 fois le nombre de coeurs dans le cluster. Avec 2 instances EC2 m5.xlarge, 8 vCore sont disponibles, ce qui sous-entend un nombre optimal de partitions compris entre __16 et 32__.  
- de faire en sorte que la taille d'une partition soit comprise entre 100 Mo et 1 Go de données non compressées. Les 22 688 fichiers jpg occupent un espace disque total de 98.4 Mo, au format 100 x 100. En supposant que l'espace occupé après redimensionnement au format 224 x 244 soit proportionnel, la nouvelle taille estimée serait de 494 Mo (98.4 x 224² / 100²), ce qui donnerait un nombre de partitions de 494 Mo / 100 Mo = 4.94 soit __5 partitions__.  

Le choix initial de 24 partitions semble cohérent vis-à-vis du nombre de coeurs, mais pour éviter d'avoir des partitions trop petites le nombre de __16 partitions__ sera retenu.  

<u>Extraction parallélisée des vecteurs de caractéristiques des images :

In [11]:
features_df = images.repartition(16).select(
    col("path"),
    col("label"),
    featurize_udf("content").alias("features")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2.5 Réduction de dimension  

La couche de sortie de modèle est de dimension 1280.  
Réduire cette dimension permettra :  
- d'accélérer les temps d'entraînement et d'inférence du modèle de classification qui sera utilisé par la suite  
- de diminuer le risque de surapprentissage (overfitting) de ce même modèle en réduisant le nombre de variables d'entrée

Les implémentations de StandardScaler et de PCA dans PySpark sont optimisées pour fonctionner sur des clusters de calcul distribué et peuvent traiter de grandes quantités de données en parallèle.  
Le code utilisé ici calcule le nombre de composantes principales nécessaires pour conserver 90 % de la variance, de manière à conserver la majorité de l'information contenue dans les caractéristiques extraites, tout en éliminant les dimensions qui contribuent le moins à la variance totale. Ce choix de 90 % pourra être ajusté en fonction des résultats du modèle de classification (performances prédictives / performances computationnelles).

### 2.5.1 Mise à l'échelle : StandardScaler  
Les table de données résilientes (Resilient Distributred Datasets ou __RDD__) sont l'un des éléments clé de Spark. Il s’agit d’un ensemble d’enregistrements ou objets d’un type spécifique, partitionnés ou distribués sur plusieurs nœuds du cluster. Les RDDs sont tolérants aux pannes : si un nœud est touché par une défaillance matérielle ou de réseau, la table résiliente est reconstruite automatiquement sur les autres nœuds et la tâche achevée.  
La principale propriété des RDDs est que Spark peut les mettre en cache mémoire (RAM) de chaque nœud. Cela permet d’économiser énormément d’accès disques et donc une amélioration significative en terme de temps de calcul.  

Les objets gérés dans les RDDs ont des types spécifiques que PySpark permet d'implémenter via le sous module `pyspark.ml.linalg` qui fait partie de l'API MLlib (Machine Learning Library). Ce module fournit des classes et des fonctions pour manipuler et représenter des vecteurs, des matrices et des opérations linéaires, qui sont utilisés dans les algorithmes de machine learning distribués de Spark.  

<u>L'implémentation de `StandardScaler` dans PySpark se fait de manière classique, mais nécessite en entrée des 'vecteurs denses' (sans données manquantes) de type `VectorUDT` :

In [12]:
# Convertion des features en format VectorUDT
to_vector_udf = udf(lambda features: Vectors.dense(features), VectorUDT())
features_df = features_df.withColumn("features", to_vector_udf("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Initialisation du StandardScaler
scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=True,
    withStd=True
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Ajustement du StandardScaler
scaler_model = scaler.fit(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Mise à l'échelle
scaled_features_df = scaler_model.transform(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2.5.2 Réduction de dimension : PCA  
Les premiers tests ont montré une reduction à 342 composantes pour conserver 90 % de la variance.  

<u>Afin de réduire les temps de traitement, la PCA est initialisée à 350 composantes :

In [16]:
# Initialisation de la PCA
pca = PCA(k=350, inputCol="scaled_features", outputCol="pca_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Ajustement du modèle PCA
pca_model = pca.fit(scaled_features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Calcul du nombre de composantes à conserver :

In [18]:
# Récupération de la variance expliquée pour chaque composante
explained_variance = pca_model.explainedVariance.toArray()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Calcul du nombre de composantes nécessaires pour expliquer au moins 90 % de la variance
cumulative_variance = np.cumsum(explained_variance)
num_components = int(np.argmax(cumulative_variance >= 0.90) + 1)

print(f"Nombre de composantes expliquant au moins 90 % de la variance : {num_components}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre de composantes expliquant au moins 90 % de la variance : 342

<u>Extraction des premières composantes :

In [20]:
# Transformation du DataFrame en utilisant le modèle PCA ajusté
pca_features_df = pca_model.transform(scaled_features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# UDF pour extraire les 'num_components' premières composantes
extract_components_udf = udf(lambda vector: Vectors.dense(vector[:num_components]), VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Application de l'UDF pour créer la colonne 'pca_features'
pca_features_df = pca_features_df.withColumn("pca_features", extract_components_udf("pca_features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Vérification par affichage des premières lignes du dataframe :

In [23]:
%%pretty
# Affichage des 5 premières lignes après suppression de la colonne 'scaled_features'
pca_features_df = pca_features_df.drop('scaled_features')
pca_features_df.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Rappel du chemin pour l'enregistrement :

In [24]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://cap9-fruits/Results_1

<u>Enregistrement au format 'parquet' :

In [25]:
pca_features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2.6 Validation du résultat

<u>Chargement des données enregistrées :

In [26]:
df_spark = spark.read.parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des premières lignes :

In [27]:
%%pretty
df_spark.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Vérification du nombre d'images traitées, et des nombres de features extraites avant et après PCA :

In [28]:
# Nombre de lignes du DataFrame = nombre d'images traitées
num_rows = df_spark.count()

# Première ligne du DataFrame
first_row = df_spark.select("features", "pca_features").first()

# Taille des vecteurs 'features' et 'pca_features'
features_size = len(first_row["features"])
pca_features_size = len(first_row["pca_features"])

# Afficher les tailles des vecteurs
print(f"Nombre d'images : {num_rows}\n")
print(f"Taille du vecteur 'features' : {features_size}\n")
print(f"Nombre de composantes expliquant 90 % de la variance : {num_components}")
print(f"Taille du vecteur 'pca_features' : {pca_features_size}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre d'images : 22688

Taille du vecteur 'features' : 1280

Nombre de composantes expliquant 90 % de la variance : 342
Taille du vecteur 'pca_features' : 342